In [ ]:
import fiona
from shapely.geometry import shape
import matplotlib.pyplot as plt
import numpy as np
import math


##############
def extract_vertices(shapefile_path):
    # Open the shapefile using fiona
    with fiona.open(shapefile_path, 'r') as polygons:
        crs = polygons.crs
        schema = polygons.schema
        Polygon_records = list(polygons)

    # Extract vertices
    vertices = []
    for polygon in Polygon_records :
        geometry = shape(polygon['geometry'])
        if geometry.geom_type == 'Polygon':
             vertices.append(list(geometry.exterior.coords))
        elif geometry.geom_type == 'MultiPolygon':
            for polygon in geometry:
                vertices.append(list(polygon.exterior.coords))

    return vertices


##################

def group_vertices_by_geometry(shapefile_path):
    vertices = extract_vertices(shapefile_path)

    # Group vertices by geometry
    grouped_vertices = []
    current_group = []
    for vertex in vertices:
        if len(current_group) == 0 or vertex[0] == current_group[-1]:
            current_group.append(vertex)
        else:
            grouped_vertices.append(current_group)
            current_group = [vertex]
    grouped_vertices.append(current_group)

    # Prepare a list to store the results
    results = []

    # Print the grouped vertices and add to results
    for i, group in enumerate(grouped_vertices):
        group_results = []
        group_results.append(f"Group {i+1}:")
        
        # Initialize dictionary to track assigned point numbers for each group
        assigned_points = {}
        
        # Sort vertices in clockwise order based on centroid
        centroid = np.mean(group, axis=0)
        sorted_vertices = sorted(group, key=lambda vertex: np.arctan2(vertex[1]-centroid[1], vertex[0]-centroid[0]))
        
        # Assign and print point numbers with vertex coordinates
        num_vertices = len(sorted_vertices)
        for j in range(num_vertices):
            vertices = sorted_vertices[j]
            num_coordinates = len(vertices)
            for k in range(num_coordinates):
                vertex = vertices[k]
                
                # Check if coordinates are already assigned a point number within the group
                if vertex in assigned_points:
                    point_number = assigned_points[vertex]
                else:
                    # Assign a new point number and update the dictionary within the group
                    point_number = len(assigned_points) + 1
                    assigned_points[vertex] = point_number
                
                result = f"Point {point_number}: {vertex[0]}, {vertex[1]}"
                group_results.append(result)

        # Add group results to overall results
        results.append(group_results)

    return results


def interpolate_points(results,interpol_distance):
    intermediate_points = []

    for group_results in results:
        for coords in group_results:
            for i in range(len(coords)-1):
                vertex = coords[i]
                next_vertex = coords[i+1]
                
                try:
                    x1, y1 = map(float, vertex)
                    x2, y2 = map(float, next_vertex)
                    
                    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
                    num_intermediate_points = int(distance /interpol_distance)

                    if num_intermediate_points > 0:
                        x_values = np.linspace(x1, x2, num=num_intermediate_points + 2)[1:-1]
                        y_values = np.linspace(y1, y2, num=num_intermediate_points + 2)[1:-1]

                        intermediate_points.append([(x, y) for x, y in zip(x_values, y_values)])
 
 
                        for point in interpolate_points:
                     # Check if the interpolated point lies on the line defined by the vertices of the grouped points
                            for i in range(len(coords) - 1):
                                vertex = coords[i]
                                next_vertex = coords[i + 1]
                                x1, y1 = vertex
                                x2, y2 = next_vertex
                                distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

                    # Check if the interpolated point lies on the line within a tolerance
                            if np.isclose(distance, 0):
                                continue  # Skip if the vertices are the same (zero distance)
                            t = ((point[0] - x1) * (x2 - x1) + (point[1] - y1) * (y2 - y1)) / (distance ** 2)
                            if 0 <= t <= 1:
                    # The interpolated point lies on the line
                               num_vertices += 1
                               result = f"Point {num_vertices}: {point[0]}, {point[1]}"
                               group_results.insert(i + 1, result)
                             
                               break  # Exit the loop since the point is already assigned to a line segment
                
                except ValueError:
                    # Skip coordinates that cannot be converted to floats
                    continue

    return  group_results

shapefile_path = '/home/mjolnir/Desktop/kada paper/20k_ building folder/'
results = group_vertices_by_geometry(shapefile_path)
interpol_distance = 2
result=interpolate_points(results,interpol_distance)

# Iterate over the results and print them
for coords in results:
      print(coords)
print()



# Iterate over the results and print them
#for group_results in results:
#   for coords in group_results:
#      print(coords)
#   print()


#def plot_grouped_vertices(grouped_vertices):
#    for group in grouped_vertices:
#        for vertex in group:
#            x, y = zip(*vertex)
#           plt.plot(x, y)

#    plt.xlabel('Longitude')
#    plt.ylabel('Latitude')
#    plt.title('Grouped Vertices Plot')
#    plt.show()

#plot_grouped_vertices(grouped_vertices)


In [ ]:

def interpolate_points(results,interpol_distance):
    intermediate_points = []

    for group_results in results:
        for coords in group_results:
            for i in range(len(coords)-1):
                vertex = coords[i]
                next_vertex = coords[i+1]
                
                try:
                    x1, y1 = map(float, vertex)
                    x2, y2 = map(float, next_vertex)
                    
                    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
                    num_intermediate_points = int(distance /interpol_distance)

                    if num_intermediate_points > 0:
                        x_values = np.linspace(x1, x2, num=num_intermediate_points )[1:-1]
                        y_values = np.linspace(y1, y2, num=num_intermediate_points )[1:-1]

                        intermediate_points.append([(x, y) for x, y in zip(x_values, y_values)])
 
 
                        for point in  intermediate_points:
                     # Check if the interpolated point lies on the line defined by the vertices of the grouped points
                            for i in range(len(coords) - 1):
                                vertex = coords[i]
                                next_vertex = coords[i + 1]
                                x1, y1 = vertex
                                x2, y2 = next_vertex
                                distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

                    # Check if the interpolated point lies on the line within a tolerance
                            if np.isclose(distance, 0):
                                continue
                            # Skip if the vertices are the same (zero distance)
                            else:  
                                t = ((point[0] - x1) * (x2 - x1) + (point[1] - y1) * (y2 - y1)) / (distance ** 2)
                                if 0 <= t <= 1:
                    # The interpolated point lies on the line
                                      num_vertices += 1
                                      result = f"Point {num_vertices}: {point[0]}, {point[1]}"
                    group_results.insert(i+1, result)
                    break  # Exit the loop since the point is already assigned to a line segment
                
                except ValueError:
                    # Skip coordinates that cannot be converted to floats
                    continue

    return  group_results

shapefile_path = '/home/mjolnir/Desktop/kada paper/20k_ building folder/'
results = group_vertices_by_geometry(shapefile_path)
interpol_distance = 0.1
result=interpolate_points(results,interpol_distance)


#Iterate over the results and print them
for coords in results:
      print(coords)
print()


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import os

file_path = '/home/mjolnir/Documents/python projects/20k_ building folder/buildings_20k_cleaned.shp'
if os.path.isfile(file_path):
    print("File exists")
else:
    print("File does not exist")

# Load the shapefile using geopandas
input_shapefile = '/home/mjolnir/Documents/python projects/20k_ building folder/buildings_20k_cleaned.shp'
data = gpd.read_file(input_shapefile)

# Create an empty set to store unique polygons
unique_polygons = set()

# Iterate over each polygon
for geom in data['geometry']:
    polygon = Polygon(geom)

    # Check if the polygon is already in the set of unique polygons
    if polygon not in unique_polygons:
        unique_polygons.add(polygon)

# Create a new GeoDataFrame with the unique polygons
unique_data = gpd.GeoDataFrame(geometry=list(unique_polygons))

# Save the result to a new shapefile
output_shapefile =  '/home/mjolnir/Documents/python projects/20k_ building folder/Unique_buildings_20k_cleaned.shp'
unique_data.to_file(output_shapefile)